In [132]:
import pdfplumber
import pandas as pd


def extract_pdf_text(pdf_path):
    # Open the PDF file
    with pdfplumber.open(pdf_path) as pdf:
        # Initialize an empty list to store the extracted data
        data = []
        # Iterate through each page
        for page_num, page in enumerate(pdf.pages):
            # Extract the text with bounding boxes
            for element in page.extract_words():
                text = element['text']
                x0, y0, x1, y1 = element['x0'], element['top'], element['x1'], element['bottom']
                data.append([page_num + 1, text, x0, y0, x1, y1])

        # Convert the list to a DataFrame
        df = pd.DataFrame(data, columns=['page', 'text', 'x0', 'y0', 'x1', 'y1'])
        return df


def find_interval(number, intervals):
    intervals = sorted(intervals)
    for interval in intervals:
        if number >= interval[0] and number < interval[1]:
            return interval
    return None


def assign_intervals_and_values(df, gridlines):
    df["interval"] = df['x0'].apply(lambda x: find_interval(x, list(gridlines.keys())))
    df["value"] = df["interval"].apply(lambda x: gridlines.get(x, None) if x else None)
    return df


def process_consecutive_values(df, target_value):
    processed_rows = []
    current_row = None

    for _, row in df.iterrows():
        if row['value'] == target_value:
            if current_row is None:
                current_row = row.copy()
            else:
                # Concatenate the text field
                current_row['text'] += row['text']
                # Update the bounding box
                current_row['x1'] = max(current_row['x1'], row['x1'])
                current_row['y1'] = max(current_row['y1'], row['y1'])
        else:
            if current_row is not None:
                processed_rows.append(current_row)
                current_row = None
            processed_rows.append(row)

    if current_row is not None:
        processed_rows.append(current_row)

    return pd.DataFrame(processed_rows)


In [133]:
def update_identifier_ranges(df, unique_identifier):
    df_identifier = df[df["value"] == unique_identifier].sort_values(by=["page","y1"]).reset_index(drop=True)
    for index in df_identifier.index:
        if index == 0:
            df_identifier.loc[index, "identifier_range_y0"] = df_identifier.loc[index, 'y0'] 
            df_identifier.loc[index, "identifier_range_y1"] = df_identifier.loc[index, 'y1']
        elif index == df_identifier.index.max():
            df_identifier.loc[index, "identifier_range_y0"] = df_identifier.loc[index-1, 'y1'] 
            df_identifier.loc[index, "identifier_range_y1"] = df_identifier.loc[index, 'y1']
        else:
            df_identifier.loc[index, "identifier_range_y0"] = df_identifier.loc[index-1, 'y1']
            df_identifier.loc[index, "identifier_range_y1"] = df_identifier.loc[index, 'y1']

    df_identifier = df_identifier[["page", "text", "identifier_range_y0", "identifier_range_y1"]]
    return df_identifier

In [134]:
def concatenate_values(df):
    new_value_column = []
    new_text_column = []

    # Iterate over the dataframe to concatenate values
    current_value = None
    current_text = ""

    for value, text in zip(df['value'], df['text']):
        if value == current_value:
            current_text += text + " "
        else:
            if current_value is not None:
                new_value_column.append(current_value)
                new_text_column.append(current_text.strip())
            current_value = value
            current_text = text + " "  # Added space at the end of each text

    # Append the last accumulated values
    if current_value is not None:
        new_value_column.append(current_value)
        new_text_column.append(current_text.strip())

    # Create a new DataFrame with the concatenated values
    new_df = pd.DataFrame({
        'value': new_value_column,
        'text': new_text_column
    })
    return new_df

def transform_df(new_df, unique_identifier):
    new_df[unique_identifier] = new_df.apply(lambda x: x.text if x.value == unique_identifier else None, axis=1).ffill()
    new_df = new_df.pivot_table(index=unique_identifier, columns='value', values='text', aggfunc='first')
    return new_df

In [135]:
council = "West Morland Barrow"
config = {
    "Bedford": {
        "gridlines": {
            (52, 121): "plate",
            (121, 220): "vrm",
            (220, 312): "make",
            (312, 418): "model",
            (418, 482): "licence_start",
            (482, 538): "licence_end"
        },
        "unique_identifier": "plate"
    },
    "BCP": {
        "gridlines": {
            (42, 76): "reference_number",
            (76, 212): "name",
            (212, 268): "vrm",
            (268, 311): "licence_number",
            (311, 364): "licence_start",
            (364, 412): "licence_end",
            (412, 528): "make",
            (528, 590): "licence_type",
        },
        "unique_identifier": "reference_number"
    },
    "Cheshire West": {
        "gridlines": {
            (52, 160): "make",
            (160, 478): "model",
            (478, 540): "vrm"
        },
        "unique_identifier": "vrm"
    },
    "East Northamptonshire": {
        "gridlines": {
            (55, 130): "vrm",
            (160, 240): "make",
            (240, 540): "model"
        },
        "unique_identifier": "vrm"
    },
    "Fylde": {
        "gridlines": {
            (18, 87): "vrm",
            (87, 216): "make",
            (216, 340): "model",
            (340, 460): "licence_start",
            (460, 540): "licence_end"
        },
        "unique_identifier": "vrm"
    },
    "Great Yarmouth": {
        "gridlines": {
            (52, 160): "vrm",
            (160, 238): "make",
            (238, 540): "model",
        },
        "unique_identifier": "vrm"
    },
    "Horsham": {
        "gridlines": {
            (52, 103): "reference_number",
            (103, 152): "licence_type",
            (152, 218): "name",
            (218, 266): "vrm",
            (266, 320): "make",
            (320, 386): "model",
            (386, 446): "licence_start",
            (446, 484): "licence_end",
            (484, 540): "plate",
        },
        "unique_identifier": "vrm"
    },
    "Kettering": {
        "gridlines": {
            (55, 102): "vrm",
            (148, 214): "make",
            (214, 540): "model"
        },
        "unique_identifier": "vrm"
    },
    "Maldon": {
        "gridlines": {
            (35, 142): "plate",
            (142, 258): "vrm",
            (258, 312): "make",
            (312, 445): "model",
            (445, 576): "colour",
            (576, 648): "seats",
            (648, 736): "licence_start",
            (736, 800): "licence_end",
        },
        "unique_identifier": "vrm"
    },
    "Milton Keynes": {
        "gridlines": {
            (256, 288): "plate",
            (288, 336): "vrm",
            (336, 406): "make",
            (406, 488): "model",
            (488, 742): "name",
            (742, 800): "licence_end"
        },
        "unique_identifier": "plate"
    },
    "North Lanarkshire": {
        "gridlines": {
            (52, 140): "vrm",
            (140, 210): "make",
            (210, 360): "model",
        },
        "unique_identifier": "vrm"
    },
    "Powys": {
        "gridlines": {
            (3, 38): "plate",
            (98, 222): "vrm",
            (222, 312): "make",
            (312, 404): "model",
            (404, 482): "licence_start",
        },
        "unique_identifier": "plate"
    },
    "Swansea": {
        "gridlines": {
            (52, 130): "vrm",
            (130, 240): "make"
        },
        "unique_identifier": "vrm"
    },
    "Wealden": {
        "gridlines": {
            (56, 98): "plate",
            (98, 136): "vrm",
            (136, 206): "make",
            (206, 272): "model",
            (272, 315): "colour",
            (315, 404): "name",
            (450, 488): "licence_start",
            (496, 540): "licence_end",
        },
        "unique_identifier": "vrm"
    },
    "West Lothian": {
        "gridlines": {
            (52, 184): "licence_type",
            (184, 295): "vrm",
            (295, 372): "make",
            (372, 440): "model",
        },
        "unique_identifier": "vrm"
    },
    "West Morland Barrow": {
        "gridlines": {
            (55, 118): "vrm",
            (118, 200): "make",
            (200, 300): "model",
        },
        "unique_identifier": "vrm"
    }
}

gridlines = config[council]["gridlines"]
unique_identifier = config[council]["unique_identifier"]

# Path to the PDF file
pdf_path = f'pdfs/no_gridlines/{council}.pdf'
df = extract_pdf_text(pdf_path)


In [136]:
df.head(50)

,page,text,x0,y0,x1,y1
0,1,Barrow,56.28000,75.24,89.00256,86.28
1,1,Reg,56.28000,89.76,72.97248,100.80
2,1,No,75.46752,89.76,88.37328,100.80
3,1,Make,119.18000,89.76,144.46160,100.80
4,1,Model,202.94000,89.76,232.08560,100.80
5,1,PK67,56.28000,104.28,79.02240,115.32
6,1,LBF,81.55056,104.28,97.31568,115.32
7,1,Toyota,119.18000,104.28,150.34592,115.32
8,1,Avensis,202.94000,104.28,236.78864,115.32
9,1,BP15,56.28000,118.80,79.25424,129.84


In [137]:
df = assign_intervals_and_values(df, gridlines)
df = process_consecutive_values(df, target_value=unique_identifier)
df_identifier = update_identifier_ranges(df, unique_identifier)
df_reduced = df[["text", "value"]].reset_index(drop=True)
new_df = concatenate_values(df_reduced)
new_df = transform_df(new_df, unique_identifier)

In [138]:
new_df

value,make,model,vrm
vrm,,,
AB03KAB,Renault,Traffic,AB03KAB
AE64VRJ,Skoda,Superb,AE64VRJ
AJ09LSN,Volkswagon,Passat Estate,AJ09LSN
AJ51MET,Skoda,Octavia,AJ51MET
AL05TAL,Skoda,Octavia,AL05TAL
...,...,...,...
YE65OSW,BMW,320,YE65OSW
YF10WUG,Ford,Mondeo,YF10WUG
YF16DXK,Skoda,Octavia,YF16DXK


In [139]:
new_df.to_csv('inspection.csv')